In [14]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from statistics import mean
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
pd.set_option('display.float_format', '{:.1f}'.format)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
%cd /Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project

/Users/macbookpro/Documents/COMP90042_NLP/Project/COMP90042_2024_Project


In [17]:
with open("data/train-claims.json", 'r') as file:
    train_claims = json.load(file)

with open("data/dev-claims.json", 'r') as file:
    dev_claims = json.load(file)

with open("data/test-claims-unlabelled.json", 'r') as file:
    test_claims_unlabelled = json.load(file)

with open("data/dev_pred.json", 'r') as file:
    dev_pred = json.load(file)

with open("data/evidence.json", 'r') as file:
    evidence = json.load(file)

with open("data/test_pred.json", 'r') as file:
    test_claims_pred = json.load(file)

In [18]:
with open("data/replaced_train.json", 'r') as file:
    replaced_train = json.load(file)

with open("data/replaced_evid.json", 'r') as file:
    replaced_evid = json.load(file)


In [6]:
with open("data/external/preprocessed_evidences.json", 'r') as file:
    prepro_evd_texts = json.load(file)

prepro_evidence = {}
i = 0
for key, value in evidence.items():
    prepro_evidence[key] = " ".join(prepro_evd_texts[i])
    i += 1

In [7]:
train_df = pd.DataFrame(columns=['claim_id','text'])
train_y = []
for key, value in train_claims.items():
    row_to_add = pd.DataFrame({
        "claim_id" :[key],
        "text": ["<sep>".join(["<cls>" +value['claim_text']] + [prepro_evidence[evd_id] for evd_id in value['evidences']])]
    })
    train_df = train_df.append(row_to_add, ignore_index = True)


In [8]:
label_list = ["SUPPORTS", "REFUTES", "NOT_ENOUGH_INFO", "DISPUTED"]
label_map = {"SUPPORTS":0, "REFUTES":1, "NOT_ENOUGH_INFO":2, "DISPUTED":3}

In [9]:
def reconstruct_data(claims,evidence):
    X = pd.DataFrame(columns=['claim_id','text','num_evidences','evidence_ids'])
    Y = []
    # prepare text: <cls> claim text <sep> evidence_1 text <sep> evidence_2 text
    for key, value in claims.items():
        row_to_add = pd.DataFrame({
            "claim_id" :[key],
            "text": ["<sep>".join(["<cls>" +value['claim_text']] + [evidence[evd_id] for evd_id in value['evidences']])],
            'num_evidences': len(value['evidences']),
            'evidence_ids': [value['evidences']]
        })
        Y.append(value['claim_label'])
        X = X.append(row_to_add, ignore_index = True)
    X_list = X['text'].values
    Y = list(map(lambda x: label_map[x], Y))
    return X, X_list, Y

In [10]:
"<sep>".join(["<cls>" +train_claims['claim-2510']['claim_text']] + [evidence[evd_id] for evd_id in train_claims['claim-2510']['evidences']])

'<cls>In 1946, PDO switched to a cool phase.<sep>There is evidence of reversals in the prevailing polarity (meaning changes in cool surface waters versus warm surface waters within the region) of the oscillation occurring around 1925, 1947, and 1977; the last two reversals corresponded with dramatic shifts in salmon production regimes in the North Pacific Ocean.<sep>1945/1946: The PDO changed to a "cool" phase, the pattern of this regime shift is similar to the 1970s episode with maximum amplitude in the subarctic and subtropical front but with a greater signature near the Japan while the 1970s shift was stronger near the American west coast.'

In [11]:
def reconstruct_X_only(claims, evidence):
    X = pd.DataFrame(columns=['claim_id','text','num_evidences','evidence_ids'])

    # prepare text: <cls> claim text <sep> evidence_1 text <sep> evidence_2 text
    for key, value in claims.items():
        row_to_add = pd.DataFrame({
            "claim_id" :[key],
            "text": ["<sep>".join(["<cls>" +value['claim_text']] + [evidence[evd_id] for evd_id in value['evidences']])],
            'num_evidences': len(value['evidences']),
            'evidence_ids': [value['evidences']]
        })
        X = X.append(row_to_add, ignore_index = True)
    X_list = X['text'].values
    
    return X, X_list

In [12]:
train_df, train_X, train_y = reconstruct_data(train_claims,evidence)
dev_df_true, dev_X_true, dev_y_true = reconstruct_data(dev_claims,evidence)

In [13]:
dev_df_pred, dev_X_pred = reconstruct_X_only(dev_pred,evidence)
test_df, test_X = reconstruct_X_only(test_claims_pred,evidence)

KeyError: 'evidences'

In [15]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.feature_extraction.text import CountVectorizer
import torch
# from keras.preprocessing.text import Tokenizer

In [16]:
tokenizer = get_tokenizer("basic_english")
train_iter = train_df['text'].tolist()

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=('<unk>', '<pad>'))
vocab.set_default_index(vocab["<unk>"])
padding_index = vocab["<pad>"]

In [17]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer, vocabulary=vocab.get_stoi(), lowercase=True)
vectorizer = CountVectorizer(tokenizer=tokenizer, vocabulary=vocab.get_stoi(), lowercase=True)

In [18]:
x_train = vectorizer.transform(train_df['text'].values).toarray()
x_dev = vectorizer.transform(dev_df_true['text'].values).toarray()
x_test = vectorizer.transform(test_df['text'].values).toarray()
x_dev_pred = vectorizer.transform(dev_df_pred['text'].values).toarray()

In [19]:
vocab_size = x_train.shape[1]
print("Vocab size =", vocab_size)
print(tokenizer(train_df['text'].values[0]))
print(x_train[0])

Vocab size = 11955
['<cls>not', 'only', 'is', 'there', 'no', 'scientific', 'evidence', 'that', 'co2', 'is', 'a', 'pollutant', ',', 'higher', 'co2', 'concentrations', 'actually', 'help', 'ecosystems', 'support', 'more', 'plant', 'and', 'animal', 'life', '.', '<sep>at', 'very', 'high', 'concentrations', '(', '100', 'times', 'atmospheric', 'concentration', ',', 'or', 'greater', ')', ',', 'carbon', 'dioxide', 'can', 'be', 'toxic', 'to', 'animal', 'life', ',', 'so', 'raising', 'the', 'concentration', 'to', '10', ',', '000', 'ppm', '(', '1%', ')', 'or', 'higher', 'for', 'several', 'hours', 'will', 'eliminate', 'pests', 'such', 'as', 'whiteflies', 'and', 'spider', 'mites', 'in', 'a', 'greenhouse', '.', '<sep>plants', 'can', 'grow', 'as', 'much', 'as', '50', 'percent', 'faster', 'in', 'concentrations', 'of', '1', ',', '000', 'ppm', 'co', '2', 'when', 'compared', 'with', 'ambient', 'conditions', ',', 'though', 'this', 'assumes', 'no', 'change', 'in', 'climate', 'and', 'no', 'limitation', 'on', 

In [20]:
sequence_pipeline = lambda x: vocab(tokenizer(x))
sequence_pipeline

<function __main__.<lambda>(x)>

In [21]:
# Get cpu, gpu or mps device for training.
# device = (
#     "cuda" if torch.cuda.is_available()
#     else "mps" if torch.backends.mps.is_available()
#     else "cpu"
# )
# print(device)

In [39]:
# set max length
max_len = 150
embedding_dim = 64
hidden_dim = 64
padding_index = vocab["<pad>"]
epochs = 20

In [23]:
import torch.nn.functional as F

def seq_collate_batch(batch):
    label_list, text_list = [], []
    for  _text, _label in batch:
        label_list.append(_label)
        text_list.append(sequence_pipeline(_text))

    label_list = torch.tensor(label_list, dtype=torch.long)
    # one_hot_labels = F.one_hot(label_list, num_classes=4)

    # Pad or truncate each sequence
    padded_sequences = []
    for seq in text_list:
        # Truncate if longer than max_len
        padded_seq = seq[:max_len]
         # Pad if shorter
        padded_seq += [padding_index] * (max_len - len(padded_seq)) 
        padded_sequences.append(torch.tensor(padded_seq))

    text_list = torch.stack(padded_sequences)

    # Stack all sequences into a single tensor
    return text_list, label_list.reshape(-1, 1)

In [24]:
from torch.utils.data import DataLoader

# Create data loaders.
xseq_train_dataloader = DataLoader(list(zip(train_X, train_y)), batch_size=5, collate_fn=seq_collate_batch)
xseq_dev_dataloader = DataLoader(list(zip(dev_X_true, dev_y_true)), batch_size=5, collate_fn=seq_collate_batch)
xseq_dev_dataloader_pred = DataLoader(list(zip(dev_X_pred, dev_y_true)), batch_size=5, collate_fn=seq_collate_batch)

# xseq_test_dataloader = DataLoader(list(zip(test_X, )), batch_size=10, collate_fn=seq_collate_batch)

In [25]:
from torch import nn

In [42]:
class SimpleLSTMNetwork(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, padding_idx):
        super().__init__()
        self.embedding  = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm_layer = nn.LSTM(embedding_dim, hidden_dim,num_layers=3, batch_first=True)

        self.forward_layer = nn.Linear(hidden_dim, 4)

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((3, batch_size, hidden_dim))#.to(device)
        c0 = torch.zeros((3, batch_size, hidden_dim))#.to(device)
        hidden = (h0, c0)
        return hidden


    def forward(self, x):
        # x = [batch size, seq length]

        embedded = self.embedding(x)
        # embedded = [batch size, seq length, emb dim]

        h0 = self.init_hidden(x.shape[0])

        output, (hidden, cell) = self.lstm_layer(embedded, h0)
        # output = [batch size, seq length, hid dim * num directions]
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]

        hidden = hidden[-1, :, :]
        # hidden = [batch size, hid dim]

        logits = torch.sigmoid(self.forward_layer(hidden))
        return logits

In [28]:
accumulate_step = 2

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # Calling .train() will evoke the tensor to start caching steps and gradients.
    model.train()
    for batch, (X, y) in enumerate(dataloader):

        # Compute prediction by directly calling the model variable.
        pred = model(X)

        y = torch.Tensor(y).flatten()
        # Calculate the loss by comparing the prediction and the true labels.
        loss = loss_fn(pred, y)

        # Backpropagation: calculate the gradient by walking back the cached steps.
        loss.backward()
        # Update the parameters of the model with the loss gradient.
        optimizer.step()
        # Remove all the gradient to be ready for the next training of the next batch.
        optimizer.zero_grad()

        if batch  == size - 1:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [29]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    # After calling eval(), the model no longer caching steps and gradient.
    # The model also does the inference faster with less resource.
    model.eval()
    test_loss, correct = 0, 0
    # This line specify that there will be no gradient in the operation below.
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            y = torch.Tensor(y).flatten()
            test_loss += loss_fn(pred, y).item()
            
            result = torch.argmax(pred,dim=1)
            correct += (result == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
print("Training LSTM network model!")

lstm_model = SimpleLSTMNetwork(vocab_size, embedding_dim, padding_index)
print(lstm_model)

# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss(torch.FloatTensor([0.2, 0.5, 0.3, 1.]))

optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
# torch.autograd.set_detect_anomaly(True)

for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(xseq_train_dataloader, lstm_model, loss_fn, optimizer)
    test(xseq_dev_dataloader_pred, lstm_model, loss_fn)

print("Done!")

print("final test:")
# test(xseq_test_dataloader, lstm_model, loss_fn)

Training LSTM network model!
SimpleLSTMNetwork(
  (embedding): Embedding(11955, 64, padding_idx=1)
  (lstm_layer): LSTM(64, 64, num_layers=5, batch_first=True)
  (forward_layer): Linear(in_features=64, out_features=4, bias=True)
)
Epoch 1
-------------------------------
Test Error: 
 Accuracy: 17.5%, Avg loss: 1.517900 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 18.2%, Avg loss: 1.519660 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 18.2%, Avg loss: 1.533606 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 29.9%, Avg loss: 1.529468 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 44.2%, Avg loss: 1.435983 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.387582 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.389148 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.389456 

Epoch 9
-------------------------

In [ ]:
# layer = 3, test error: 36.4%
